In [1]:
!pwd
file = 'column_description.csv'

/home/kylemeador/symdesign/dependencies/python


In [2]:
column_description_dict = {}
with open(file, 'r') as f:
    for pair in f.readlines():
        pair = pair.strip().split(',')
        column_description_dict[pair[0]] = pair[1]

In [3]:
column_description_dict

{'': '',
 'buns_heavy_total': 'Buried unsaturated hydrogen bonds between heavy atoms in the pose',
 'buns_hpol_total': 'Buried unsaturated hydrogen bonds between hydrogen atoms in the pose',
 'buns_total': 'Buried unsaturated hydrogen bonds in the pose',
 'contact_count': 'Number of carbon carbon contacts across interface residues',
 'cst_weight': 'Total weight of constraints used to keep the pose from moving in 3D space',
 'fsp_energy': 'Total weight of sequence constraints used to favor certain amino acids in design. Only some protocols have values',
 'fsp_total_stability': 'fsp_energy + total pose energy',
 'full_stability': 'total pose energy (essentially REU from above)',
 'int_area_hydrophobic': 'solvent accessible surface total interface area hydrophobic',
 'int_area_polar': 'solvent accessible surface total interface area polar',
 'int_area_res_summary_hydrophobic_A_oligomer': 'Sum of each interface residues individual area for oligomer A - hydrophobic',
 'int_area_res_summary_

In [1]:
import pandas as pd
import numpy as np
import math
from itertools import product, combinations
data_file = '/home/kylemeador/symdesign/P432/All_Scores/3l8r_1ho1-DEGEN_1_1-ROT_36_1-tx_4_Trajectories.csv'
df = pd.read_csv(data_file, index_col=0, header=[0,1])

In [25]:
def datafame_permutation_test(grouped_df, diff_df, group1_samples=None, compare='mean', permutations=1000):
    """From a two group dataframe, compare  run a permutation test and compare """
    # permut_array = [grouped_df.iloc[:group1_samples, :].ave().sub(
    #     grouped_df.iloc[:group1_samples, :].ave()) for i in range(permutations)]
    permut_array = []
    for i in range(permutations):
        shuffled_df = grouped_df.sample(n=len(grouped_df))
#         print(getattr(grouped_df.iloc[:group1_samples, :], compare)())
#         print(getattr(grouped_df.iloc[group1_samples:, :], compare)())
        
        # permut_array.append(grouped_df.iloc[:group1_samples, :].ave().sub(grouped_df.iloc[:group1_samples, :].ave()))
        permut_array.append(getattr(shuffled_df.iloc[:group1_samples, :], compare)().sub(getattr(shuffled_df.iloc[group1_samples:, :], compare)()))
#         permut_array.append(grouped_df.iloc[:group1_samples, :].getattr(compare)().sub(
#             grouped_df.iloc[:group1_samples, :].getattr(compare)()))

#     difference_pvalue_df = pd.DataFrame()
#     bool_df = pd.DataFrame(ignore_index=True)
#     for permut in permut_array:
#         bool_df.append(abs(permut).ge(abs(diff_df)))
#     difference_pvalue_df = bool_df.mean()

    abs_df = diff_df.abs()
    # how many times the absolute permuted set is less than the absolute difference set to return a value of true. 
    # When taking the mean of these the probability will be reflected
    bool_df = pd.DataFrame([permut.abs().lt(abs_df) for permut in permut_array])

    return bool_df.mean()
#     for column in list(diff_df.index):  #.columns       
#         greater_count = 0
#         # check how many permutations have a greater value than the measured difference
#         for permut in permut_array:
#             if abs(permut[column]) > abs(diff_df[column]):
#                 greater_count += 1
#         difference_pvalue_df[column] = greater_count/permutations
#         difference_pvalue[column] = min(greater_count/permutations, 1 - greater_count/permutations)
# 
#     return difference_pvalue_df

In [40]:
protocols = list(df[('protocol', 'protocol')].unique())
# protocols.remove('protocol')
protocols.remove(np.nan)
# print(protocols)
# display(df)
simple_df =df.copy()
simple_df.columns = df.columns.droplevel(0)
pvalue_df = pd.DataFrame()
for protocol_pair in combinations(protocols, 2):
#     print(protocol_pair)
    select_df = simple_df[(simple_df.protocol == protocol_pair[0]) | (simple_df.protocol == protocol_pair[1])]  # ['protocol']
    difference_df = select_df.groupby('protocol').mean()
    difference_df = difference_df.iloc[0, :].sub(difference_df.iloc[1, :])
#     select_df = simple_df.drop(simple_df[(simple_df.protocol != protocol_pair[0]) | (simple_df.protocol != protocol_pair[1])].index)  # ['protocol']
#     display(select_df)
#     group_df = simple_df.groupby(['protocol']).filter(lambda x: x[''] in protocol_pair)
#     df.groupby(('protocol', 'protocol'))
#     display(group_df)
#     select_df = group_df[('protocol', 'protocol') in protocol_pair]
#     display(select_df)
    group_length = len(select_df[select_df['protocol'] == protocol_pair[0]])
    
#     pvalue_df[pd.MultiIndex.from_tuples(protocol_pair)] = datafame_permutation_test(select_df, difference_df, group1_samples=group_length)
    pvalue_df[protocol_pair] = datafame_permutation_test(select_df, difference_df, group1_samples=group_length)    

KeyError: "None of [MultiIndex([('f', 'a', 'v', 'o', 'r', '_', 'p', 'r', 'o', 'f', 'i', 'l', 'e'),\n            ('l', 'i', 'm', 'i', 't', '_', 't', 'o', '_', 'p', 'r', 'o', 'f')],\n           )] are in the [columns]"

In [36]:
test_df = pd.DataFrame(pvalue_df.T['buns_heavy_total'])
# test_df = pd.DataFrame(pvalue_df.loc['buns_heavy_total', :])
display(test_df)

,buns_heavy_total
"(favor_profile, limit_to_profile)",0.396
"(favor_profile, no_constraint)",0.000
"(favor_profile, combo_profile)",0.000
"(limit_to_profile, no_constraint)",0.000
"(limit_to_profile, combo_profile)",0.000
"(no_constraint, combo_profile)",0.000


In [27]:
display(pvalue_df)

,"(favor_profile, limit_to_profile)","(favor_profile, no_constraint)","(favor_profile, combo_profile)","(limit_to_profile, no_constraint)","(limit_to_profile, combo_profile)","(no_constraint, combo_profile)"
buns_heavy_total,0.396,0.000,0.000,0.000,0.000,0.0
buns_hpol_total,0.396,0.000,0.000,0.000,0.000,0.0
buns_total,0.396,0.000,0.000,0.000,0.000,0.0
contact_count,0.949,0.562,0.836,0.334,0.000,0.0
cst_weight,0.757,0.755,0.903,0.000,0.000,0.0
fsp_energy,0.949,0.857,0.836,0.000,0.000,0.0
fsp_total_stability,0.832,0.426,0.836,0.334,0.000,0.0
full_stability,0.710,0.699,0.836,0.334,0.000,0.0
int_area_hydrophobic,0.317,0.857,0.000,0.677,0.323,0.0
int_area_polar,0.707,0.857,0.836,0.343,0.653,0.0


### First round values

In [13]:
display(pvalue_df)

,"(favor_profile, limit_to_profile)","(favor_profile, no_constraint)","(favor_profile, combo_profile)","(limit_to_profile, no_constraint)","(limit_to_profile, combo_profile)","(no_constraint, combo_profile)"
buns_heavy_total,1.0,1.0,1.0,1.0,1.0,1.0
buns_hpol_total,1.0,1.0,1.0,1.0,1.0,1.0
buns_total,1.0,1.0,1.0,1.0,1.0,1.0
contact_count,0.0,1.0,0.0,1.0,1.0,1.0
cst_weight,1.0,0.0,0.0,1.0,1.0,1.0
fsp_energy,0.0,0.0,0.0,1.0,1.0,1.0
fsp_total_stability,1.0,0.0,0.0,1.0,1.0,1.0
full_stability,1.0,0.0,0.0,1.0,1.0,1.0
int_area_hydrophobic,0.0,0.0,1.0,0.0,0.0,1.0
int_area_polar,1.0,0.0,0.0,0.0,0.0,1.0


In [6]:
protocols = list(df[('protocol', 'protocol')].unique())
# protocols.remove('protocol')
protocols.remove(np.nan)
# print(protocols)
# display(df)
for protocol_pair in product(protocols, protocols):
#     print(protocol_pair)
    group_df = df.groupby(('protocol', 'protocol'))
    df.groupby(('protocol', 'protocol'))
    display(group_df)
    select_df = group_df[('protocol', 'protocol') in protocol_pair]
    display(select_df)

('favor_profile', 'favor_profile')


KeyError: 'Column not found: False'

In [58]:
protocols = list(df[('protocol', 'protocol')].unique())
# protocols.remove('protocol')
protocols.remove(np.nan)
# print(protocols)
# display(df)
simple_df =df.copy()
# simple_df.columns = df.columns.droplevel(0)
pvalue_df = pd.DataFrame()
for protocol_pair in combinations(protocols, 2):
#     print(protocol_pair)
    select_df = simple_df[(simple_df[('protocol', 'protocol')] == protocol_pair[0]) | (simple_df[('protocol', 'protocol')] == protocol_pair[1])]  # ['protocol']
    difference_df = select_df.groupby(('protocol', 'protocol')).mean()
    difference_df = difference_df.iloc[0, :].sub(difference_df.iloc[1, :])
#     select_df = simple_df.drop(simple_df[(simple_df.protocol != protocol_pair[0]) | (simple_df.protocol != protocol_pair[1])].index)  # ['protocol']
#     display(select_df)
#     group_df = simple_df.groupby(['protocol']).filter(lambda x: x[''] in protocol_pair)
#     df.groupby(('protocol', 'protocol'))
#     display(group_df)
#     select_df = group_df[('protocol', 'protocol') in protocol_pair]
#     display(select_df)
    group_length = len(select_df[select_df[('protocol', 'protocol')] == protocol_pair[0]])
    pvalue_df[(protocol_pair, 'similarity')] = datafame_permutation_test(select_df, difference_df, group1_samples=group_length)

In [59]:
pvalue_mean_df = pvalue_df.mean()
display(pvalue_mean_df)

((favor_profile, limit_to_profile), similarity)    0.530229
((favor_profile, no_constraint), similarity)       0.504343
((favor_profile, combo_profile), similarity)       0.453486
((limit_to_profile, no_constraint), similarity)    0.309629
((limit_to_profile, combo_profile), similarity)    0.171771
((no_constraint, combo_profile), similarity)       0.000000
dtype: float64

In [60]:
pvalue_calc_df = pvalue_df.copy()

In [62]:
pvalue_df = pvalue_calc_df.copy()
pvalue_df.columns = pd.MultiIndex.from_tuples(pvalue_df.columns)
pvalue_df.columns = pvalue_df.columns.droplevel(1)
pvalue_df.columns = pd.MultiIndex.from_tuples(pvalue_df.columns)
pvalue_df.columns = pd.MultiIndex.from_product([pvalue_df.columns, ['similarity']])
display(pvalue_df)

NotImplementedError: isna is not defined for MultiIndex

In [48]:
# pvalue_df.columns = pd.MultiIndex.from_tuples(pvalue_df.columns)
add_df = pd.concat([pvalue_df], axis=1, keys=['similarity']).swaplevel(0, 2, 1)
display(add_df)

limit_to_profile  \
                                                    favor_profile   
                                                       similarity   
pose buns_heavy_total                                       0.408   
     buns_hpol_total                                        0.408   
     buns_total                                             0.408   
     contact_count                                          0.964   
     cst_weight                                             0.765   
     fsp_energy                                             0.964   
     fsp_total_stability                                    0.857   
     full_stability                                         0.725   
     int_area_hydrophobic                                   0.325   
     int_area_polar                                         0.718   
     int_area_res_summary_hydrophobic_A_oligomer            0.125   
     int_area_res_summary_hydrophobic_B_oligomer            0.609   
     int_area_res_summary_polar_A_oligomer                  0.756   
     int_area_res_summary_polar_B_oligomer                  0.524   
     int_area_res_summary_total_A_oligomer                  0.656   
     int_area_res_summary_total_B_oligomer                  0.825   
     int_area_total                                         0.339   
     int_connectivity_A                                     0.864   
     int_connectivity_B                                     0.804   
     int_energy_context_A_oligomer                          0.108   
     int_energy_context_B_oligomer                          0.342   
     int_energy_context_complex                             0.865   
     int_energy_context_delta                               0.067   
     int_energy_context_oligomer                            0.380   
     int_energy_res_summary_A_oligomer                      0.075   
     int_energy_res_summary_B_oligomer                      0.344   
     int_energy_res_summary_complex                         0.897   
     int_energy_res_summary_delta                           0.080   
     int_energy_res_summary_oligomer                        0.380   
     int_separation                                         0.337   
     interaction_energy_complex                             0.249   
     number_hbonds                                          0.244   
     ref                                                    0.976   
     rmsd                                                   0.590   
     shape_complementarity                                  0.405   

                                                 no_constraint combo_profile  \
                                                 favor_profile favor_profile   
                                                    similarity    similarity   
pose buns_heavy_total                                    0.000         0.000   
     buns_hpol_total                                     0.000         0.000   
     buns_total                                          0.000         0.000   
     contact_count                                       0.582         0.881   
     cst_weight                                          0.778         0.933   
     fsp_energy                                          0.856         0.881   
     fsp_total_stability                                 0.446         0.881   
     full_stability                                      0.736         0.881   
     int_area_hydrophobic                                0.856         0.000   
     int_area_polar                                      0.856         0.881   
     int_area_res_summary_hydrophobic_A_oligomer         0.002         0.475   
     int_area_res_summary_hydrophobic_B_oligomer         0.041         0.042   
     int_area_res_summary_polar_A_oligomer               0.439         0.313   
     int_area_res_summary_polar_B_oligomer               0.585         0.722   
     int_area_res_summary_total_A_oligomer               0.655    

In [65]:
s1 = pd.Series(['a', 'b'], name='fun')
s2 = pd.Series(['c', 'd'])
s3 = pd.concat([s1, s2])
display(s1)

0    a
1    b
Name: fun, dtype: object

In [71]:
d1 = pd.DataFrame([s1, s2])
display(d1)

,0,1
fun,a,b
Unnamed 0,c,d
